In [1]:
import json
import os
from datetime import datetime
import json
import os

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix

import src.data.Dataset as dt

In [2]:
required_paths = ["/ai4eutils", "/CameraTraps", "/yolov5"]
python_path = os.environ.get("PYTHONPATH", "")
root_path = os.getcwd()

for path in required_paths:
    if not any(p.endswith(path) for p in python_path.split(":")):
        python_path += f":{root_path}/data/external{path}"

os.environ["PYTHONPATH"] = python_path

!echo "PYTHONPATH: $PYTHONPATH"

PYTHONPATH: :/Users/carlos/WORKSPACE/MegaClassifier/data/external/ai4eutils:/Users/carlos/WORKSPACE/MegaClassifier/data/external/CameraTraps:/Users/carlos/WORKSPACE/MegaClassifier/data/external/yolov5


In [3]:
initial_threshold = float(0.02)
steps_threshold = float(0.005)

positives_coberture = 0.0

OUTPUT_PATH = os.path.abspath("./resources/json")

MODEL_PATH = os.path.abspath("./models/MegaDetector/md_v5a.0.0.pt")
# MODEL_PATH = os.path.abspath("./models/MegaDetector/md_v5b.0.0.pt")
# MODEL_PATH = os.path.abspath("./models/MegaDetector/md_v4.1.0.pb")

DATASET_PATH = os.path.abspath("./dataset/emptyNonEmptyDatasetCROP")

OUTPUT_FILE_PATH = os.path.join(OUTPUT_PATH, "detector_results.json")
CHECKPOINT_PATH = os.path.join(OUTPUT_PATH, "checkpoint.json")
CHECKPOINT_FREQ = 5

TRAIN_CSV = os.path.abspath("./data/interim/train/20Train.csv")

REPORT_CSV = os.path.abspath("./reports/model_coberture")

os.makedirs(OUTPUT_PATH, exist_ok=True)

print(f"MODEL_PATH: {MODEL_PATH}")
print(f"OUTPUT_FILE_PATH: {OUTPUT_FILE_PATH}")
print(f"OUTPUT_PATH: {OUTPUT_PATH}")
print(f"TRAIN_CSV: {TRAIN_CSV}")

MODEL_PATH: /Users/carlos/WORKSPACE/MegaClassifier/models/MegaDetector/md_v5a.0.0.pt
OUTPUT_FILE_PATH: /Users/carlos/WORKSPACE/MegaClassifier/resources/json/detector_results.json
OUTPUT_PATH: /Users/carlos/WORKSPACE/MegaClassifier/resources/json
TRAIN_CSV: /Users/carlos/WORKSPACE/MegaClassifier/data/interim/train/20Train.csv


In [4]:
threshold = round(initial_threshold + steps_threshold, 3)
load_from_checkpoint = True
while positives_coberture != 1.0:
    threshold = round(threshold - steps_threshold, 3)

    detection_inicialitation_time = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")

    if load_from_checkpoint:
        command = f'python data/external/CameraTraps/detection/run_detector_batch.py "{MODEL_PATH}" "{DATASET_PATH}" "{OUTPUT_FILE_PATH}" --recursive --threshold "{threshold}" --checkpoint_path "{CHECKPOINT_PATH}" --checkpoint_frequency "{CHECKPOINT_FREQ}" --resume_from_checkpoint "{CHECKPOINT_PATH}"'
    else:
        command = f'python data/external/CameraTraps/detection/run_detector_batch.py "{MODEL_PATH}" "{DATASET_PATH}" "{OUTPUT_FILE_PATH}" --recursive --threshold "{threshold}" --checkpoint_path "{CHECKPOINT_PATH}" --checkpoint_frequency "{CHECKPOINT_FREQ}"'
    os.system(command)

    with open(OUTPUT_FILE_PATH, "r") as file:
        data = json.load(file)

    for image in data["images"]:
        image["file"] = (
            image["file"]
            .replace(
                OUTPUT_FILE_PATH,
                "",
            )
            .replace("/", "\\")
        )

    info = {
        "detection_initial_time": detection_inicialitation_time,
        "detection_completion_time": data["info"]["detection_completion_time"],
        "format_version": data["info"]["format_version"],
        "detector": data["info"]["detector"],
        "detector_threshold": threshold,
        "detector_metadata": data["info"]["detector_metadata"],
    }

    final_output = {
        "images": data["images"],
        "detection_categories": data["detection_categories"],
        "info": info,
    }

    threshold_str = str(threshold).replace(".", "_")
    json_name = f"{len(data['images'])}_images_{threshold_str}_threshold.json"
    model_name = os.path.basename(MODEL_PATH).split(".")[0]
    tmp_path = os.path.join(OUTPUT_PATH, model_name)
    os.makedirs(tmp_path, exist_ok=True)
    NEW_OUTPUT_FILE_PATH = os.path.join(tmp_path, json_name)

    with open(NEW_OUTPUT_FILE_PATH, "w") as f:
        json.dump(final_output, f, indent=1)

    os.remove(OUTPUT_FILE_PATH)

    data = final_output

    dataset = dt.load_from_csv(TRAIN_CSV)

    report_columns = [
        "file_name",
        "label",
        "threshold",
        "detector_label",
        "false_positive",
        "false_negative",
        "time_inference",
    ]
    report = pd.DataFrame(columns=report_columns)

    model = data["info"]["detector"].split(".")[0]

    for image in data["images"]:
        image_file = image["file"]
        indexes = dataset[dataset["file_name"] == image_file]

        if len(indexes) == 1:
            label = int(indexes["label"].iloc[0])
            detector_label = 1 if image["max_detection_conf"] > 0.0 else 0
            false_positive = int(label == 0 and detector_label == 1)
            false_negative = int(label == 1 and detector_label == 0)
            time_inference = image["time_inference"]

            new_row = {
                "file_name": image_file,
                "label": label,
                "threshold": data["info"]["detector_threshold"],
                "detector_label": detector_label,
                "false_positive": false_positive,
                "false_negative": false_negative,
                "time_inference": time_inference,
            }
            report = pd.concat([report, pd.DataFrame([new_row])], ignore_index=True)

    report_name = f"{len(data['images'])}_images_{model}.csv"
    REPORT_CSV_FILE = os.path.join(REPORT_CSV, report_name)

    if os.path.exists(REPORT_CSV_FILE):
        existing_report = dt.load_from_csv(REPORT_CSV_FILE)
        new_report = pd.concat([existing_report, report], ignore_index=True)
        dt.dataset_to_csv(new_report, REPORT_CSV_FILE)
    else:
        dt.dataset_to_csv(report, REPORT_CSV_FILE)

    positivos_reales = report["label"].sum()
    positivos_cubiertos = report[
        (report["label"] == 1) & (report["detector_label"] == 1)
    ].shape[0]

    positives_coberture = (
        (positivos_cubiertos / positivos_reales) if positivos_reales > 0 else 0
    )
    
    load_from_checkpoint = False
    
    print()
    print("-----------------------------------------------------------------------")
    print(f"El porcentaje de positivos cubiertos es {positives_coberture*100:.2f}%")
    print("-----------------------------------------------------------------------")
    print()


Restored 5 entries from the checkpoint
60 image files found in the input directory
The checkpoint file will be written to /Users/carlos/WORKSPACE/MegaClassifier/resources/json/checkpoint.json
PyTorch reports 0 available CUDA devices
GPU available: False
Using PyTorch version 1.10.1


Fusing layers... 
Model summary: 574 layers, 139990096 parameters, 0 gradients
  0%|          | 0/60 [00:00<?, ?it/s]

Loaded model in 1.52 seconds
Loaded model in 1.52 seconds
Bypassing image /Users/carlos/WORKSPACE/MegaClassifier/dataset/emptyNonEmptyDatasetCROP/leporido/conejo_Paco_Carro_JC/PICT0646 (2).JPG
Bypassing image /Users/carlos/WORKSPACE/MegaClassifier/dataset/emptyNonEmptyDatasetCROP/leporido/Conejo_PNC_2014_JC/c8663IM000121.JPG
Bypassing image /Users/carlos/WORKSPACE/MegaClassifier/dataset/emptyNonEmptyDatasetCROP/vaca/cow_CaltechCameraTrap/585c030e-23d2-11e8-a6a3-ec086b02610b.jpg
Bypassing image /Users/carlos/WORKSPACE/MegaClassifier/dataset/emptyNonEmptyDatasetCROP/vaca/vaca_Paco_Carro_JC/PICT0042.JPG
Bypassing image /Users/carlos/WORKSPACE/MegaClassifier/dataset/emptyNonEmptyDatasetCROP/vaca/COW_ZOO_5_6_7_9/21_20210418 (1498).JPG
Processing image /Users/carlos/WORKSPACE/MegaClassifier/dataset/emptyNonEmptyDatasetCROP/jabali/jabali_PNM_2013-O_ISAAC/f27560IMAG0113.JPG


 10%|█         | 6/60 [00:02<00:23,  2.27it/s]

Processing image /Users/carlos/WORKSPACE/MegaClassifier/dataset/emptyNonEmptyDatasetCROP/jabali/WILDBOAR_ZOO_5_6_7_9/20_20210219 (2902) (2021_04_15 06_16_22 UTC).JPG


...

Processing image /Users/carlos/WORKSPACE/MegaClassifier/dataset/emptyNonEmptyDatasetCROP/cervidae/ciervo_simone_2/34_20201024 (5494).JPG


 28%|██▊       | 17/60 [00:44<01:47,  2.51s/it]

Processing image /Users/carlos/WORKSPACE/MegaClassifier/dataset/emptyNonEmptyDatasetCROP/cervidae/ciervo_simone_2/37_20201024 (1156).JPG


 30%|███       | 18/60 [00:46<01:43,  2.46s/it]

Processing image /Users/carlos/WORKSPACE/MegaClassifier/dataset/emptyNonEmptyDatasetCROP/cervidae/CERVIDREDORFALLOWDEER_ZOO_1_4/34_20201024 (2742).JPG
